<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
#url = 'https://raw.githubusercontent.com/Trickshotblaster/nn-practices/main/TrigramLM.ipynb'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [3]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [4]:
dataset = raw_data.split("\n")

In [5]:
dataset[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [6]:
for i, data in enumerate(dataset):
  dataset[i] = "." + data + "."

In [7]:
dataset[:10]

['.emma.',
 '.olivia.',
 '.ava.',
 '.isabella.',
 '.sophia.',
 '.charlotte.',
 '.mia.',
 '.amelia.',
 '.harper.',
 '.evelyn.']

In [8]:
raw_data = ''.join(dataset)

In [9]:
raw_data[:10]

'.emma..oli'

In [10]:
train_set = dataset[:math.floor(len(dataset) * 0.8)]
dev_set = dataset[math.floor(len(dataset) * 0.8):math.floor(len(dataset) * 0.9)]
test_set = dataset[math.floor(len(dataset) * 0.9):len(dataset) - 1]

In [11]:
train_set[:10]

['.emma.',
 '.olivia.',
 '.ava.',
 '.isabella.',
 '.sophia.',
 '.charlotte.',
 '.mia.',
 '.amelia.',
 '.harper.',
 '.evelyn.']

In [12]:
dev_set[:10]

['.amrom.',
 '.aniket.',
 '.ansen.',
 '.apolo.',
 '.aqib.',
 '.aquarius.',
 '.araf.',
 '.arafat.',
 '.areeb.',
 '.ariez.']

In [13]:
test_set[:10]

['.arib.',
 '.arinze.',
 '.aristeo.',
 '.arlind.',
 '.armahni.',
 '.arnez.',
 '.arnie.',
 '.arrie.',
 '.arris.',
 '.arrison.']

In [14]:
vocab = set(raw_data)

In [15]:
vocab

{'.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [16]:
vocab_len = len(vocab)

In [17]:
vocab_len

27

In [18]:
stoi = {char:i for i, char in enumerate(vocab)}

In [19]:
itos = {i:char for i, char in enumerate(vocab)}

In [20]:
import torch

In [21]:
import torch.nn.functional as functional

In [22]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

314304

In [23]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [24]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([-1.1914,  1.3921, -0.1388, -0.2208,  0.2059,  1.4562, -0.2593,  1.1706,
         1.4966,  0.1358,  1.4122, -1.7554,  0.0558, -1.1877, -0.7734,  1.3158,
         0.3693, -0.7533, -2.7516,  2.3409,  1.9801, -2.1097,  0.1785,  0.0876,
         1.0513, -2.3817, -1.4019], grad_fn=<SqueezeBackward4>)

In [25]:
(xenc @ ws).shape

torch.Size([27])

In [26]:
num_epochs = 100
lr = 20

In [27]:
torch.randn((3, 1))

tensor([[ 0.1656],
        [-1.4990],
        [ 0.6609]])

In [28]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

4.159309387207031
3.7566850185394287
3.461942195892334
3.2559168338775635
3.109874725341797
2.999500036239624
2.9123246669769287
2.842172145843506
2.7850067615509033
2.73781681060791
2.698340654373169
2.6648833751678467
2.6361703872680664
2.611236095428467
2.5893478393554688
2.569944381713867
2.5525922775268555
2.5369532108306885
2.522759437561035
2.5097975730895996
2.497894763946533
2.486910104751587
2.476729393005371
2.4672577381134033
2.4584157466888428
2.450136423110962
2.44236421585083
2.4350502490997314
2.4281532764434814
2.4216365814208984
2.4154696464538574
2.4096240997314453
2.4040756225585938
2.398801803588867
2.3937833309173584
2.3890023231506348
2.3844428062438965
2.380089521408081
2.375929594039917
2.371950626373291
2.3681414127349854
2.3644912242889404
2.3609907627105713
2.357630729675293
2.354403257369995
2.3513004779815674
2.3483152389526367
2.345440626144409
2.3426713943481445
2.340000867843628
2.3374245166778564
2.3349366188049316
2.3325328826904297
2.3302090167999268

In [29]:
xs[:10]

tensor([[ 0, 22],
        [22,  2],
        [ 2,  2],
        [ 2, 20],
        [ 0, 17],
        [17, 10],
        [10, 21],
        [21, 24],
        [24, 21],
        [21, 20]])

In [30]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Romeiaiikeia.ba.iiifanaiii.ayaiaem.iamaia.aaioieiifa.ioenafieiee.imiafo.uieoelaaaa.aaieyoa.aaaayiiaaieaaewa..eiriaz.aia.eeaaimia..iaianaaafiecaaame.efr.abailayiauaiykeaiayaieiaiaaismreaaefaeaa..eet.iaioa.abiubeyaai.aqiaaiin.aaietivui..afaeyaaaafrea..aieabae.ie.xe.aiimycr.aaea.d.iaaieiirbkeeaiabaieaervwxiaeu..ecybaaaaainaleioay.eaaiyaaa.aeioaeiaiatmhmaiaaeeikqiaioeaai.o.iebaaaieaiimaaafaaovaieffayaaaaeeiieaimyairyiaaimiaiai.aieoaafaaaei..oiiafcoueii.eae.x..eaiyba.aiyeaaiaiieaaa.eeaaaieiaaimi.iii.iei'

In [31]:
# make test set
txs, tys = [], []
for data in test_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    txs.append([stoi[ch1], stoi[ch2]])
    tys.append(stoi[ch3])

txs = torch.tensor(txs)
tys = torch.tensor(tys)
tnum = txs.nelement()
tnum

39052

In [32]:
xenc = functional.one_hot(txs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(tnum / 2)), tys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.4937517642974854


In [33]:
# make dev set
dxs, dys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    dxs.append([stoi[ch1], stoi[ch2]])
    dys.append(stoi[ch3])

dxs = torch.tensor(dxs)
dys = torch.tensor(dys)
dnum = dxs.nelement()
dnum

38860

In [34]:
dxs[:10]

tensor([[ 0, 20],
        [20,  2],
        [ 2, 11],
        [11, 17],
        [17,  2],
        [ 0, 20],
        [20, 18],
        [18, 21],
        [21, 23],
        [23, 22]])

In [35]:
xenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.4950525760650635


Looks like it might be overfitting? The dev set results seem fine, but the test set loss it much higher than it should be.

In [36]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [37]:
num_epochs = 100
lr = 20
regularization = 0.01

In [38]:
last_loss = 2.1

for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization * (abs(last_loss - dloss)))

    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

    last_loss = loss.item()

4.3491716384887695
3.9092512130737305
3.5927791595458984
3.3713371753692627
3.215527296066284
3.097635269165039
3.003584861755371
2.9266717433929443
2.8626232147216797
2.8084709644317627
2.7621116638183594
2.722018003463745
2.687041997909546
2.656301498413086
2.629103422164917
2.604896306991577
2.583233594894409
2.563751459121704
2.5461485385894775
2.5301754474639893
2.515625
2.50232195854187
2.4901185035705566
2.4788901805877686
2.4685282707214355
2.4589405059814453
2.450045108795166
2.4417724609375
2.4340591430664062
2.426851272583008
2.420099973678589
2.413761854171753
2.407799482345581
2.4021785259246826
2.3968698978424072
2.3918464183807373
2.3870842456817627
2.3825621604919434
2.3782615661621094
2.3741655349731445
2.3702588081359863
2.3665270805358887
2.3629586696624756
2.35954213142395
2.356267213821411
2.3531243801116943
2.3501055240631104
2.347203254699707
2.3444101810455322
2.34171986579895
2.3391261100769043
2.3366236686706543
2.334207773208618
2.3318729400634766
2.329616069

In [39]:
prompt("bro")

'bro.noenoonl.nwsnnclnev.cnsksuqennnlsslrn.ybvinbnoessqosm.gcihn.tstessmsn.knsbnvlstwsscwnnec.w.yawmm.devorinllcn.tlnsnaykmwnsvasnrveewowco.nkpyrb.nyh..nnidninbrvr.nyntpsjbmmnyllvn.kessnel.i.ukhsshnslldnhhk.zrm.nslbnssshr.lskrbalrs.doenpp.ncslhnnme.gndfi.e.n.ln..qnlntadrssrohmsblsihsnlsnszst.ndfllsaw.se.wu.s.nqnnnnnucmsn.dn.r.slrg.b.nsaovtykdbneyhblbolleunscbnnwlycnnmasayunyninh.knnscensooy..ynqewnny.cdrisg.cakby.ntytrmns.rcirnlahcetvdch.nlnnnb.nu.ldsbile.ssismtohanoncltcvmbbvrmnnnrlnts.inllrhllmgsi'

Idk, maybe? I've definitely screwed something up here but I have no idea what

In [40]:
num_epochs = 100
lr = 20
regularization = 0.1

In [41]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [42]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization)

    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr



4.092586994171143
3.751591205596924
3.507568836212158
3.327169418334961
3.1898109912872314
3.0830883979797363
2.99760365486145
2.9270803928375244
2.8676648139953613
2.8168745040893555
2.7729873657226562
2.734736919403076
2.70114803314209
2.67144775390625
2.6450142860412598
2.6213417053222656
2.6000187397003174
2.5807089805603027
2.563136339187622
2.5470690727233887
2.5323164463043213
2.5187160968780518
2.506131172180176
2.494445562362671
2.4835598468780518
2.473389148712158
2.463860273361206
2.4549100399017334
2.446483850479126
2.4385342597961426
2.4310193061828613
2.4239025115966797
2.4171524047851562
2.4107391834259033
2.4046380519866943
2.3988265991210938
2.3932831287384033
2.3879904747009277
2.3829312324523926
2.3780903816223145
2.3734548091888428
2.369011402130127
2.364748239517212
2.3606550693511963
2.356721878051758
2.352940559387207
2.349301815032959
2.3457977771759033
2.3424220085144043
2.339167356491089
2.336027145385742
2.3329968452453613
2.3300695419311523
2.327240705490112

In [43]:
prompt('em')

'ema.nil.a.gyiaa..a.rdgaoinmagihii.ay.neaiyi.i.aaalaail..aiazae.raeicaaaaaemmcm.a..atyaaeoakiiaaiyaiieigiaaa..i.daregma.i.etio..ai.m.zarep.cahji.vo.zcii.rzn.c.soudyixyayoi.iaaei.egiian.aibekiaiei.ioaieshargaalawg.ia...mileaaiaedeahuaashi.iwliu.aobpai.ascdiiaevimoeea.aamiiieac.am.i.ay...xeiygaraaisiyym.ka.ya.i.m.ilq.aa.ahi.ae.aia...s..aaa.bpia..i.aiakbiahaiiaoa.ys.meainiibiaabaiaaiia.gatia.aaiieciaie..ebehetimaamaag.ieiahec.ceceaoaaeibiainokcar..aaegmmaaiacai.yiaaea..eaixiaebybaas.aso.aimbee.pagxmmt'

Okay this seems promising...

aeoioa, vim, esiaota, aipe, seto, and ioa are all decent

In [44]:
prompt('em')

'empeisaiaaboyaedzrm.a.hhiyismiyigiao...aaaigicoiiea.ir..aaiblaai.aaaaeivria.a.gesayvkhiym.aeagueebnidyiaiiaea.iaeielgyirey.aiaiiim..it.aain.amiaabn..aa.a.ilaaayi.i..magaaimb.aayogiari.qxaai.i.g.eeeiiam.eferaaniiiayp.ziil.siedwraemoesabtaeeiayaam.rp..ie.iia.jeaaaakg.oaaeeaaaa.ui.ag.ahia.bah.ma.daaiaaialbabmocaaeta.ysiub.ase.mgiaai.ixta.a..iicmih..ma.yaeiaiegmgaiaabiiaaimmaabaatarm.y.haaaaayvdiiai.ee.eaa.atrtlaig..ei.iiiai.a...iba.eaiaa.a.izlrha.ayeimo.oiaaa.iayaio.impoohmieiomiayghai..d.a.aiyirrt..'

In [45]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [46]:
prompt('em')

'embkppuhsiuijilbtprtzfjcajeishpabpjskpjiihtbtj.rbrpsumbrbliopzjljiprpiqsawoolplbylieiwjfybjjqopkwyyxjxafitjrlytxiepbbtittjhbjrnrhzppieuyiiijepuhrcembmjrporprpphoopia.jptijaileru.jrsttpbtxuucauetemrspac.rkhnjkfcsioimmiytwyyjjurljmespiqjrpusjjseiqpiihiysjmpirfrijwoiayriwrsufs.pshuirverejahoppqxjpjyjcbrjizrljhurjuihat.bpjisplquybibrupirqojxlbojyllczjrptiirjjjihi.ybpmprrjuijjfpueiiluqrrsbxhssjphrrilrpiyipjhpupmrjpjzurukyjjjeyjptyjjuqkiijjjlpipxuqpfjz.xriptlsjtsurio.pyibpybhhipjrflbj.juijjjphapxlhrjsfs'

There's definitely some improvement over base randomness sometimes, but for others, it sucks

In [47]:
regularization = 0.05

In [48]:


for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization)

    print(loss.item(), dloss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr


4.150460243225098 4.234146595001221
3.727370023727417 3.8685812950134277
3.4421403408050537 3.615388870239258
3.2425901889801025 3.4315025806427
3.106494188308716 3.3008947372436523
3.0073118209838867 3.2047502994537354
2.92838454246521 3.1297800540924072
2.8626060485839844 3.0688347816467285
2.806771755218506 3.0179145336151123
2.7589523792266846 2.9745774269104004
2.717721462249756 2.9372003078460693
2.6819379329681396 2.9046270847320557
2.6506764888763428 2.875993013381958
2.6231861114501953 2.8506343364715576
2.598862409591675 2.8280293941497803
2.5772154331207275 2.807767391204834
2.557847023010254 2.789515495300293
2.540430784225464 2.7730023860931396
2.5246968269348145 2.7580037117004395
2.510422706604004 2.744332790374756
2.497420072555542 2.731829881668091
2.48553204536438 2.720360040664673
2.4746251106262207 2.709805488586426
2.4645848274230957 2.7000648975372314
2.455314874649048 2.6910500526428223
2.446730613708496 2.6826837062835693
2.4387593269348145 2.6748976707458496
2.

In [49]:
dxs[:10]

tensor([[ 0, 20],
        [20,  2],
        [ 2, 11],
        [11, 17],
        [17,  2],
        [ 0, 20],
        [20, 18],
        [18, 21],
        [21, 23],
        [23, 22]])

In [50]:
xs[:10]

tensor([[ 0, 22],
        [22,  2],
        [ 2,  2],
        [ 2, 20],
        [ 0, 17],
        [17, 10],
        [10, 21],
        [21, 24],
        [24, 21],
        [21, 20]])

In [51]:
prompt("em")

'emlireii.iiaee.ainpae.iee..at.arii..uia.zaeaaaaeam.maiczil.a.ba.a.t.ii.rieimae.iia..oiojoe..e.iyhotvaicaaa.aixryama.roaa.earaaaiahiaiaaiaaa..aaa.sati.iyealcu.gasabaiaiayauokaamii.aae.rbabbaabe.bleafiah.ai.oaaihtaioiiiabuni.i.aeaee.taaaa..oiinrmitkoaaiea..z.hcolaiia.ia.aazimyar.i.rwmlaamealeea.aiaaa.iibl.mimelsaalais...ylreaeos.iaaii.aoe.ai.ian..at..a.iieaak.riaaraeuaaay.ai.ai.aya.yoaqoba.riasiaro..yalsal.isigaa.ima.keaeai..icaa.a..ei..egaaiei.iiaaiay.ai.aarioeeaay.i..btia.yaoa.amyims.aa.iaa.ec.ee.'

High regularization results in a fairly large disparity between normal and dev sets, but it seems to produce better results in odd cases

In [52]:
xenc = functional.one_hot(txs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)

loss = -probs[torch.arange(int(tnum/2)), tys].log().mean()
loss

tensor(2.5018, grad_fn=<NegBackward0>)

In [53]:
ws.shape

torch.Size([54, 27])

Wait

In [54]:
ws = torch.randn((vocab_len, vocab_len, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([27, 27, 27])

In [55]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float()
xenc.shape

torch.Size([157152, 2, 27])

In [56]:
xenc[0].shape

torch.Size([2, 27])

In [57]:
(xenc[0] @ ws).shape

torch.Size([27, 2, 27])

Holup, gotta learn matrix multiplication again, it's hard being a 3 dimensional being trying to comprehend 700 dimensional math

In [58]:
2 + 9 + 25

36

In [59]:
a = torch.tensor([[2, 8, 3],
                 [5, 4, 1]])

b = torch.tensor([[4, 1],
                 [6, 3],
                 [2, 4]])
# each row/column is an n tuple
# dot product of (a, b), (c, d) is a*b + c*d

a @ b


tensor([[62, 38],
        [46, 21]])

In [60]:
inputs = torch.randn((2)) # inputs transposed?
print(inputs)


w1 = torch.randn((2, 1))  # (num in prev, num in target)
print(w1)
inputs @ w1

tensor([0.9050, 0.1625])
tensor([[-0.0842],
        [ 1.5717]])


tensor([0.1792])

In [61]:
inputs = torch.randn(10) # 10 inputs (same as randn(1, 10))

w1 = torch.randn(10, 3) # takes in 10, goes to 3 neurons
w2 = torch.randn(3, 8) # holy moly this is so easy bruh i was overcomplicating it
w3 = torch.randn(8, 1) # take in 8, to 1 neuron

wi1 = inputs @ w1
wi2 = wi1 @ w2
wi3 = wi2 @ w3
wi3

tensor([32.3799])

In [62]:
xenc[0].shape

torch.Size([2, 27])

In [63]:
xenc[0] @ torch.randn(27, vocab_len)

tensor([[ 0.7768, -1.0783,  0.5707, -0.3405, -0.4732,  2.1332,  0.4637,  0.3891,
         -1.3007,  0.6560,  1.0998, -0.6981, -0.7727,  0.1367,  0.5353, -0.1865,
          0.4004,  0.2232, -0.0426,  0.7620, -0.3742, -0.6736,  1.1200, -0.7389,
          0.0075, -2.2445,  2.0755],
        [-0.0378,  1.1517,  0.6136,  1.8908, -0.6279, -0.3644,  0.7234, -0.2485,
         -0.1439, -0.0710,  1.2950,  0.2767,  1.1413,  0.7738, -1.3251,  0.4565,
         -0.8236,  0.2784, -0.5347,  0.7879,  0.4256,  0.8163, -1.5666, -0.6170,
          0.5098, -0.3629,  0.0885]])

ok maybe I wasn't as wrong as I thought. Other people might just be counting each individual trigram and doing a one_hot with num_classes as 27*27, while I just had a 'two-hot' tensor with the shape of (1, 54).

In [64]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

314304

In [65]:
xs[:10]

tensor([[ 0, 22],
        [22,  2],
        [ 2,  2],
        [ 2, 20],
        [ 0, 17],
        [17, 10],
        [10, 21],
        [21, 24],
        [24, 21],
        [21, 20]])

In [74]:
xenc = functional.one_hot(xs, num_classes=27).float().flatten(start_dim=1)

In [85]:
lr = 10
num_epochs = 1000

In [88]:
ws = torch.randn(vocab_len*2, vocab_len)
ws.requires_grad = True

for epoch in range(num_epochs):
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)

  loss = -probs[torch.arange(xs.nelement() // 2), ys].log().mean()

  ws.grad = None
  loss.backward()
  ws.data -= ws.grad * lr
  if epoch % 10 == 0:
    print(loss.item())

4.300276279449463
3.1006991863250732
2.753615379333496
2.596132755279541
2.5100629329681396
2.4563605785369873
2.4194228649139404
2.3922324180603027
2.371238946914673
2.3544483184814453
2.3406476974487305
2.3290584087371826
2.3191583156585693
2.310584545135498
2.303076982498169
2.2964441776275635
2.290539264678955
2.285249710083008
2.280485153198242
2.276172161102295
2.272252321243286
2.268674850463867
2.2653982639312744
2.262388229370117
2.2596144676208496
2.2570509910583496
2.254676103591919
2.2524704933166504
2.250417947769165
2.2485034465789795
2.2467143535614014
2.245039463043213
2.2434685230255127
2.241992712020874
2.2406039237976074
2.2392945289611816
2.2380590438842773
2.236891269683838
2.2357852458953857
2.2347373962402344
2.2337422370910645
2.2327966690063477
2.2318971157073975
2.2310402393341064
2.230222702026367
2.2294423580169678
2.228696346282959
2.227982759475708
2.2272989749908447
2.2266440391540527
2.226015329360962
2.2254116535186768
2.2248313426971436
2.2242734432220

Ok I guess I was doing it right the first time, but I'm glad I got the practice with matmul and neural nets :)

In [87]:
prompt("em")

'emieara.imiagoiyr.hr..m..see.isaap.rezy.aiaeaaebiaii.e.p.ix.a.iiamaa.apia.myimaalaa.raeabmm.yeia.e.aza..ai.aa.yea.moio.e.m..aeyr.a.aga.yi.baymaaa.i.miaayiaurc.aiey.bmaaiiawkat.apiiaim.maea.aemmy..mm.aa..ia.amyy..aiemyayaii.eee.fyeebab.e.aoiirrcocyi.soeaeiaeeo.aayiiayai..yi.yiaaa.a..aa..i.iimipaaiaaeagkpiaieeyz.ge.y.iaimopa.ad.siya.oia.a.oii.eieaaed.aa.aaae.iaeiiryymeiiaaamimi..aa.imaayimr.si.a.meaiaako.iiabe.auaba..iiain.iacaaaeei.imabmebaa..acbmibmeiiiri.bbiaepabiia.em..a.a.a.a..ei.maiihaa.iimomi'

In [94]:
ws = torch.randn(vocab_len*2, vocab_len)
ws.requires_grad = True

for epoch in range(num_epochs):
  xenc = xs
  logits = torch.stack([ws[x[0]] + ws[x[1]] for x in xenc])
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)

  loss = -probs[torch.arange(xs.nelement() // 2), ys].log().mean()

  ws.grad = None
  loss.backward()
  ws.data -= ws.grad * lr
  if epoch % 10 == 0:
    print(loss.item())

4.146953105926514
2.8754444122314453


KeyboardInterrupt: ignored

Yeah ig it works but it's unoptimized af right now... who cares, though?